### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#installation step
!pip install transformers
!pip install t5
!pip install sentencepiece
!pip install bpemb

     |████████████████████████████████| 645kB 3.3MB/s 
     |████████████████████████████████| 3.8MB 10.9MB/s 
     |████████████████████████████████| 1.0MB 15.1MB/s 
     |████████████████████████████████| 890kB 19.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9ba0c66645692d6a9e6c368f76aa6082cf15da4353ca4b6b7deb66f7025ca9c1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 143kB 3.5MB/s 
     |████████████████████████████████| 8.3MB 57.0MB/s 
     |████████████████████████████████| 3.3MB 49.6MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 296kB 60.8MB/s 
     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 3.9MB 49.3MB/s 
     |████████████████████████████████| 450kB 60.5MB/s 
  Created wheel for gast: filename

In [2]:
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

--2020-05-11 00:44:11--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 74.125.69.138, 74.125.69.102, 74.125.69.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.69.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ums068j8r195s8d0e1kac2q4a367b8hs/1589157825000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download [following]
--2020-05-11 00:44:11--  https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ums068j8r195s8d0e1kac2q4a367b8hs/1589157825000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download
Resolving doc-10-2s-docs.googleusercontent.com (doc-10-2s-docs.googleusercontent.com)... 74.125.202.132, 2607:f8b0:4001:c06::84
Connecting to doc-10-2s-docs.googleusercontent.com (doc-10-2s-docs.

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
from torch.optim import SGD
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools

import sentencepiece as spm
from bpemb import BPEmb

import transformers
import t5
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split


In [4]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf

BASE_DIR = "gs://t5at" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tensorflow_gcs_config.configure_gcs_from_colab_auth('/device:CPU:0')
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)


Installing dependencies...
Setting up GCS access...
Running on TPU: grpc://10.49.75.2:8470
Instructions for updating:
non-resource variables are not supported in the long term


##Initial Loading from Pickle

In [0]:
VOCAB_SIZE = 5000

In [6]:
ls data/AD_NMT-master

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_MSA_English_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_MSA_English_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_MSA_English_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [8]:
#few dataset examples
print(data_MSA_English_both[0:5])
print(data_LAV_MSA_both[0:5])

#print length of train + val dataset, print length of both (train + val) + test
print(len(data_MSA_English_trainval))
print(len(data_MSA_English_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [9]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
msa_en_train, msa_en_val = train_test_split(data_MSA_English_trainval, test_size=.2)
msa_en_test = data_MSA_English_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [11]:
print(len(msa_en_train))
print(len(msa_en_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def list_to_csv(ds, src='msa', trg='en', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + 'csv'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as csv:
        for i, arr in enumerate(ds):
            csv.write(arr[1] + ',' + arr[0] + '\n')

In [0]:
list_to_csv(msa_en_train, 'msa', 'en', 'train')
list_to_csv(msa_en_val, 'msa', 'en', 'val')
list_to_csv(msa_en_test, 'msa', 'en', 'test')

list_to_csv(lav_msa_train, 'lav', 'msa', 'train')
list_to_csv(lav_msa_val, 'lav', 'msa', 'val')
list_to_csv(lav_msa_test, 'lav', 'msa', 'test')

list_to_csv(mag_msa_train, 'mag', 'msa', 'train')
list_to_csv(mag_msa_val, 'mag', 'msa', 'val')
list_to_csv(mag_msa_test, 'mag', 'msa', 'test')

In [41]:
!gsutil -m cp -r "data/*" $DATA_DIR

Copying file://data/val/val_msa_en.csv [Content-Type=text/csv]...
Copying file://data/val/val_mag_msa.csv [Content-Type=text/csv]...
Copying file://data/train/train_mag_msa.csv [Content-Type=text/csv]...
Copying file://data/train/train_msa_en.csv [Content-Type=text/csv]...
Copying file://data/train/train_lav_msa.csv [Content-Type=text/csv]...
Copying file://data/AD_NMT-master/english-Arabic-test.pkl [Content-Type=application/octet-stream]...
Copying file://data/AD_NMT-master/Magribi_MSA-test.pkl [Content-Type=application/octet-stream]...
Copying file://data/AD_NMT-master/english-Arabic-both.pkl [Content-Type=application/octet-stream]...
Copying file://data/AD_NMT-master/english-Arabic-train.pkl [Content-Type=application/octet-stream]...
Copying file://data/AD_NMT-master/Magribi_MSA-train.pkl [Content-Type=application/octet-stream]...
Copying file://data/AD_NMT-master/LAV-MSA-2-test.pkl [Content-Type=application/octet-stream]...
Copying file://data/AD_NMT-master/LAV-MSA-2-train.pkl [Con

##Loading MultiBPemb: Multilingual BPEmb Sentencepiece Model

In [67]:
multibpemb = BPEmb(lang="multi", vs=1000000, dim=300)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


##Tensor Processing + Add to TaskRegistry

### English to Arabic Task

In [0]:
msa_en_split_csv_path = {
    "train": os.path.join(DATA_DIR, "train/train_msa_en.csv"),
    "validation": os.path.join(DATA_DIR, "val/val_msa_en.csv")
}
msa_en_example_count = {
    "train": 7200,
    "validation": 1800
}

In [69]:
def msa_en_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(msa_en_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(msa_en_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'\xd9\x85\xd9\x86 \xd8\xa7\xd9\x84\xd9\x85\xd9\x85\xd9\x83\xd9\x86 \xd8\xa7\xd9\x84\xd8\xa7 \xd8\xaa\xd8\xa7\xd8\xaa\xd9\x8a \xd8\xba\xd8\xaf\xd8\xa7', 'target': b'She may not come here tomorrow'}
{'source': b'\xd8\xa7\xd9\x86\xd8\xaa\xd8\xa8\xd9\x87! \xd9\x87\xd9\x86\xd8\xa7\xd9\x83 \xd8\xad\xd9\x81\xd8\xb1\xd8\xa9 \xd9\x83\xd8\xa8\xd9\x8a\xd8\xb1\xd8\xa9 \xd9\x87\xd9\x86\xd8\xa7\xd9\x83', 'target': b"Watch out! There's a big hole there"}
{'source': b'\xd8\xa7\xd8\xb9\xd9\x8a\xd8\xaf\xd9\x87 \xd8\xa7\xd9\x84\xd9\x8a', 'target': b'Bring it back to me'}
{'source': b'\xd8\xaf\xd8\xa7\xd8\xa6\xd9\x85\xd8\xa7 \xd9\x85\xd8\xa7 \xd8\xa7\xd8\xb5\xd8\xa7\xd8\xaf\xd9\x81 \xd8\xb3\xd9\x8a\xd8\xa7\xd8\xad \xd9\x8a\xd8\xa7\xd8\xa8\xd8\xa7\xd9\x86\xd9\x8a\xd9\x8a\xd9\x86 \xd9\x81\xd9\x8a \xd9\x87\xd8\xb0\xd9\x87 \xd8\xa7\xd9\x84\xd9\x85\xd9\x86\xd8\xb7\xd9\x82\xd8\xa9 ', 'target': b'We often come across Japanese tourists in this area'}
{'source': b'\xd8\xaf\xd8\xae\xd9\x84\xd8\xaa \xd9\

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def msa_en_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate MSA to English: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [71]:
filepath = str(multibpemb.model_file)
print(filepath)

/root/.cache/bpemb/multi/multi.wiki.bpe.vs1000000.model


In [0]:
t5.data.TaskRegistry.remove("msa_en_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "msa_en_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=msa_en_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[msa_en_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path= filepath, #str(multibpemb.model_file), #t5.data.DEFAULT_SPM_PATH
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)

###Levantine to MSA Task

In [0]:
lav_msa_split_csv_path = {
    "train": os.path.join(DATA_DIR, "train/train_lav_msa.csv"),
    "validation": os.path.join(DATA_DIR, "val/val_lav_msa.csv")
}
lav_msa_example_count = {
    "train": 11044,
    "validation": 2761
}

In [74]:
def lav_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(lav_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(lav_msa_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'\xd8\xaa\xd9\x85\xd8\xab\xd9\x84 \xd9\x81\xd9\x8a\xd9\x87 \xd8\xac\xd9\x88\xd8\xa7\xd9\x87\xd8\xb1', 'target': b'\xd8\xa8\xd8\xaa\xd9\x85\xd8\xab\xd9\x84 \xd9\x81\xd9\x8a\xd9\x87 \xd8\xac\xd9\x88\xd8\xa7\xd9\x87\xd8\xb1'}
{'source': b'\xd9\x81\xd9\x87\xd9\x88 \xd8\xa7\xd8\xb0\xd8\xa7\xd8\xb5\xd8\xa7\xd8\xb1 \xd8\xb4\xd8\xb1\xd8\xb9\xd9\x8a\xd8\xa7 \xd9\x88 \xd9\x85\xd8\xb4\xd8\xb1\xd9\x88\xd8\xb9\xd8\xa7', 'target': b'\xd9\x87\xd9\x88 \xd8\xb5\xd8\xa7\xd8\xb1 \xd9\x84\xd9\x83\xd9\x86 \xd8\xb4\xd8\xb1\xd8\xb9\xd9\x8a \xd9\x88 \xd9\x85\xd8\xb4\xd8\xb1\xd9\x88\xd8\xb9'}
{'source': b'\xd8\xad\xd9\x82\xd8\xa7  \xd8\xb9\xd9\x82\xd8\xaf \xd9\x84\xd8\xaf\xd9\x89 \xd8\xa7\xd9\x84\xd8\xae\xd9\x88\xd8\xa7\xd8\xb5', 'target': b'\xd8\xb9\xd9\x86\xd8\xac\xd8\xaf \xd8\xb9\xd9\x82\xd8\xaf \xd8\xb9\xd9\x86\xd8\xaf \xd8\xa7\xd9\x84\xd8\xae\xd8\xa7\xd8\xb5'}
{'source': b'\xd9\x84\xd9\x83\xd9\x86 \xd9\x81\xd9\x8a \xd9\x85\xd8\xaf\xd8\xb1\xd8\xb3\xd8\xa9 \xd9\x88\xd8\xb7\xd9\x86\xd9\x8a\xd8\xa

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def lav_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate Levantine to MSA: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("lav_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "lav_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=lav_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[lav_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path= filepath, #str(multibpemb.model_file), #t5.data.DEFAULT_SPM_PATH
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)

###Maghrib to MSA Task

In [0]:
mag_msa_split_csv_path = {
    "train": os.path.join(DATA_DIR, "train/train_mag_msa.csv"),
    "validation": os.path.join(DATA_DIR, "val/val_mag_msa.csv")
}
mag_msa_example_count = {
    "train": 14188,
    "validation": 3548
}

In [78]:
def mag_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(mag_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(mag_msa_translation_dataset_fn("train").take(5)):
    print(example)
    print(example['source'].decode())
    print(example['target'].decode())
    print(len(example['source']))
    print(len(example['target']))

{'source': b' \xd8\xa7\xd9\x86\xd9\x87\xd8\xa7 \xd9\x85\xd8\xb9\xd9\x84\xd9\x85\xd8\xa9 \xd9\x88\xd8\xa7\xd8\xa6\xd9\x84 \xd8\xaa\xd8\xaf\xd8\xb1\xd8\xb3\xd9\x87 \xd8\xa7\xd9\x84\xd8\xb9\xd9\x84\xd9\x88\xd9\x85    ', 'target': b'\xd9\x87\xd9\x8a \xd9\x85\xd8\xb9\xd9\x84\xd9\x85\xd8\xa9 \xd8\xaf\xd9\x8a\xd8\xa7\xd9\x84 \xd9\x88\xd8\xa7\xd8\xa6\xd9\x84 \xd9\x83\xd8\xaa\xd9\x82\xd8\xb1\xd9\x8a\xd9\x87 \xd8\xb3\xd9\x8a\xd8\xa7\xd9\x86\xd8\xb3'}
 انها معلمة وائل تدرسه العلوم    
هي معلمة ديال وائل كتقريه سيانس
57
57
{'source': b' \xd9\x83\xd8\xa7\xd9\x86\xd8\xaa \xd8\xaa\xd9\x88\xd8\xac\xd8\xaf \xd8\xb4\xd8\xac\xd8\xb1\xd8\xa9 \xd8\xaa\xd9\x8a\xd9\x86 \xd8\xaa\xd8\xb9\xd8\xb7\xd8\xb1 \xd9\x83\xd9\x84 \xd8\xa7\xd9\x84\xd8\xad\xd9\x8a   ', 'target': b'\xd9\x83\xd8\xa7\xd9\x86\xd8\xaa \xd9\x88\xd8\xa7\xd8\xad\xd8\xaf \xd8\xb4\xd8\xac\xd8\xb1\xd8\xa9 \xd8\xa7\xd9\x84\xd9\x83\xd8\xb1\xd9\x85\xd9\x88\xd8\xb5 \xd9\x83\xd8\xaa\xd8\xb9\xd8\xb7\xd8\xb1 \xd8\xa7\xd9\x84\xd8\xad\xd9\x88\xd9\x85\xd8\xa9

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def mag_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate Maghrib to Arabic: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("mag_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "mag_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=mag_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[mag_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path= filepath, #str(multibpemb.model_file), #t5.data.DEFAULT_SPM_PATH
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)

##Dataset Mixture

In [0]:
t5.data.MixtureRegistry.remove("ar_translation")
t5.data.MixtureRegistry.add(
    "ar_translation",
    ["msa_en_translation", "lav_msa_translation", "mag_msa_translation"],
     default_rate=1.0
)

##Fine Tune

In [0]:
MODEL_SIZE = "base" #@param["small", "base", "large"]
# Public GCS path for T5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE)

model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 128, "targets": 128},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)


In [87]:
STEPS = 1000000 #@param {type: "integer"}
MODEL_SIZE = "base"
model.train(
    mixture_or_task_name="ar_translation",
    steps=STEPS,
    )


INFO:tensorflow:Using config: {'_model_dir': 'gs://t5at/models/base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.49.75.2:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.49.75.2:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.49.75.2:8470', '_evaluation_master': 'grpc://10.49.75.2:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': 

UnknownError: ignored

##Evaluation

In [65]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = train_batch_size * 4
model.eval(
    mixture_or_task_name="ar_translation",
    checkpoint_steps="all"
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://t5at/models/base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.49.75.2:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.49.75.2:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.49.75.2:8470', '_evaluation_master': 'grpc://10.49.75.2:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': 

##Predictions

In [66]:
msa_en_1 = "احبك" #@param {type:"string"}
msa_en_2 = "ماذا تقولون" #@param {type:"string"}
msa_en_3 = "انا في بيتي" #@param {type:"string"}
msa_en_4 = "مرحبا" #@param {type:"string"}

questions = [msa_en_1, msa_en_2, msa_en_3, msa_en_4]

now = time.time()
# Write out the supplied questions to text files.
predict_inputs_path = os.path.join(MODEL_DIR, "predict_inputs_%d.txt" % now)
predict_outputs_path = os.path.join(MODEL_DIR, "predict_outputs_%d.txt" % now)
# Manually apply preprocessing by prepending "triviaqa question:".
with tf.io.gfile.GFile(predict_inputs_path, "w") as f:
  for q in questions:
    f.write("Translate MSA to English: %s\n" % q.lower())

# Ignore any logging so that we only see the model's answers to the questions.
with tf_verbosity_level('ERROR'):
  model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
  model.predict(
      input_file=predict_inputs_path,
      output_file=predict_outputs_path,
      # Select the most probable output token at each step.
      temperature=0,
  )

# The output filename will have the checkpoint appended so we glob to get 
# the latest.
prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
with tf.io.gfile.GFile(prediction_files[-1]) as f:
  for q, a in zip(questions, f):
    if q:
      print("Q: " + q)
      print("A: " + a)
      print()


Predictions using checkpoint 1000900:

Q: احبك
A: I'm going


Q: ماذا تقولون
A: I'm not hungry


Q: انا في بيتي
A: I'm not going to go


Q: مرحبا
A: I'm going


